In [44]:
import os
import json
import GEOparse
import pandas as pd
from openai import OpenAI

from prompts.column_description import system_message, dataset_columns_description_prompt, dataset_input
from utils import utils
from utils import openai_utils
import prompts
from importlib import reload

from dotenv import load_dotenv

load_dotenv()

True

In [45]:
reload(utils)
reload(openai_utils)
reload(prompts.column_description)

<module 'prompts.column_description' from '/home/bek/Desktop/iCog/semantic parsing/Bio_data_semantic_parsing/prompts/column_description.py'>

In [46]:
gse= GEOparse.get_GEO(geo="GSE12277", destdir="./data/GEO")

11-Mar-2025 10:09:49 DEBUG utils - Directory ./data/GEO already exists. Skipping.
11-Mar-2025 10:09:49 INFO GEOparse - File already exist: using local version.
11-Mar-2025 10:09:49 INFO GEOparse - Parsing ./data/GEO/GSE12277_family.soft.gz: 
11-Mar-2025 10:09:49 DEBUG GEOparse - DATABASE: GeoMiame
11-Mar-2025 10:09:49 DEBUG GEOparse - SERIES: GSE12277
11-Mar-2025 10:09:49 DEBUG GEOparse - PLATFORM: GPL570


/home/bek/Desktop/iCog/semantic parsing/Bio_data_semantic_parsing/venv/lib/python3.12/site-packages/GEOparse/GEOparse.py:401: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  return read_csv(StringIO(data), index_col=None, sep="\t")
11-Mar-2025 10:09:55 DEBUG GEOparse - SAMPLE: GSM308237
11-Mar-2025 10:09:55 DEBUG GEOparse - SAMPLE: GSM308239
11-Mar-2025 10:09:55 DEBUG GEOparse - SAMPLE: GSM308241
11-Mar-2025 10:09:55 DEBUG GEOparse - SAMPLE: GSM308242
11-Mar-2025 10:09:56 DEBUG GEOparse - SAMPLE: GSM308244
11-Mar-2025 10:09:56 DEBUG GEOparse - SAMPLE: GSM308245
11-Mar-2025 10:09:57 DEBUG GEOparse - SAMPLE: GSM308246
11-Mar-2025 10:09:57 DEBUG GEOparse - SAMPLE: GSM308247
11-Mar-2025 10:09:57 DEBUG GEOparse - SAMPLE: GSM308254
11-Mar-2025 10:09:58 DEBUG GEOparse - SAMPLE: GSM308255
11-Mar-2025 10:09:58 DEBUG GEOparse - SAMPLE: GSM308317
11-Mar-2025 10:09:58 DEBUG GEOparse - SAMPLE: GSM308461
11-Mar-2025 10:09:59 DEBUG GEOparse - SAMP

### Column description generation for the table in GSMs

In [47]:
gsm= gse.gsms["GSM308246"]
data= gsm.table
data.head(-5)

,ID_REF,VALUE,ABS_CALL,DETECTION P-VALUE
0,AFFX-BioB-5_at,115.457000,P,0.000581
1,AFFX-BioB-M_at,119.770000,P,0.000044
2,AFFX-BioB-3_at,84.816300,P,0.000095
3,AFFX-BioC-5_at,296.351000,P,0.000052
4,AFFX-BioC-3_at,470.652000,P,0.000044
...,...,...,...,...
54665,1570631_at,12.766800,P,0.023926
54666,1570632_at,1.179860,A,0.780518
54667,1570633_at,0.159784,A,0.953857
54668,1570635_at,1.926470,A,0.753906


In [48]:
sample = data.sample(20)

In [49]:
df_string = sample.to_string()
print(df_string)


             ID_REF        VALUE ABS_CALL  DETECTION P-VALUE
1555      201966_at   425.018000        P           0.001221
18670   219245_s_at    61.952100        P           0.004150
8736    209181_s_at   492.437000        P           0.000244
38575     238713_at     6.738500        A           0.149658
10085     210556_at    15.380500        P           0.000244
41398     241536_at     9.263680        M           0.046143
46883    1555111_at     3.856630        A           0.466064
10140     210614_at     6.659910        A           0.567627
13103     213662_at    10.278300        A           0.129639
46495  1554592_a_at    16.563600        P           0.013184
15693     216260_at    11.668700        P           0.030273
48638    1557709_at     0.660489        A           0.969727
24153     224272_at     8.703000        A           0.334473
8732      209177_at   169.831000        P           0.000732
9004      209450_at    79.336600        P           0.000244
10955   211487_x_at  428

In [50]:
dataset_description= "GEO is a public functional genomics data repository supporting MIAME-compliant data submissions. Array- and sequence-based data are accepted. Tools are provided to help users query and download experiments and curated gene expression profiles."

In [51]:
generation_params = {
    'temperature': 0.0,
    'max_tokens': 4096,
}

In [52]:
messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': dataset_columns_description_prompt 
                                + dataset_input.format(dataset_description= dataset_description, dataset= df_string)}
]

In [53]:
response = openai_utils.openai_generate(messages=messages, **generation_params)
descriptions = utils.postprocess_response(response.choices[0].message.content)

In [54]:
descriptions

{'ID_REF': {'description': 'Unique identifier for each gene expression profile in the dataset.',
  'data_type': 'string'},
 'VALUE': {'description': 'The expression level of the gene, a higher value indicates a higher expression level.',
  'data_type': 'float'},
 'ABS_CALL': {'description': "The absolute call for each gene expression profile, indicating the quality of the measurement. 'P' stands for Present, 'A' for Absent, and 'M' for Marginal.",
  'data_type': 'string'},
 'DETECTION P-VALUE': {'description': 'The p-value from the detection algorithm, indicating the confidence of the measurement. A lower value indicates a higher confidence.',
  'data_type': 'float'}}

### FOL predicate generation for table entries

In [13]:
gsm_id = gsm.metadata.get("geo_accession", ["Unknown_GSE"])[0]
print(gsm_id)

GSM308246


Generate a unique ID for each row in a GSM

In [14]:

# Create a unique ID by combining row index and GSM ID
data.insert(0, "Unique_ID", data.index.astype(str) + "_" + gsm_id)

# Display the updated DataFrame
print(data.head())

     Unique_ID          ID_REF     VALUE ABS_CALL  DETECTION P-VALUE
0  0_GSM308246  AFFX-BioB-5_at  115.4570        P           0.000581
1  1_GSM308246  AFFX-BioB-M_at  119.7700        P           0.000044
2  2_GSM308246  AFFX-BioB-3_at   84.8163        P           0.000095
3  3_GSM308246  AFFX-BioC-5_at  296.3510        P           0.000052
4  4_GSM308246  AFFX-BioC-3_at  470.6520        P           0.000044


In [15]:
sample_data=data.sample(1000)
print(sample_data) 

             Unique_ID        ID_REF      VALUE ABS_CALL  DETECTION P-VALUE
34838  34838_GSM308246   234976_x_at   1.211110        A           0.805420
37395  37395_GSM308246     237533_at   1.368080        A           0.888428
33285  33285_GSM308246   233421_s_at   1.012710        A           0.753906
18008  18008_GSM308246     218582_at  85.563000        P           0.000244
8131    8131_GSM308246     208572_at   3.355460        A           0.432373
...                ...           ...        ...      ...                ...
34263  34263_GSM308246     234401_at   3.608660        A           0.194580
44463  44463_GSM308246     244601_at   0.890298        A           0.850342
45534  45534_GSM308246  1553301_a_at   6.807730        A           0.274170
42414  42414_GSM308246   242552_x_at  14.148600        A           0.398926
53568  53568_GSM308246  1568690_a_at   4.216260        A           0.665527

[1000 rows x 5 columns]


In [16]:
columns = list(sample_data.columns)
print("Extracted Columns:", columns)


Extracted Columns: ['Unique_ID', 'ID_REF', 'VALUE', 'ABS_CALL', 'DETECTION P-VALUE']


In [17]:
prompt = f"""
    You are a data standardization expert. Convert the following dataset column names into standardized predicate names:
    {columns}
    Return a JSON dictionary where keys are original column names and values are standardized predicate names.
    """

In [18]:
messages =[
    {'role':'system', 'content':"You are an expert in data standardization."},
    {'role':'user', 'content':prompt}
]

In [19]:
response = openai_utils.openai_generate(messages=messages, **generation_params)
predicate_mapping = response.choices[0].message.content

In [20]:
print("Predicate Mapping:", predicate_mapping)

Predicate Mapping: {
    "Unique_ID": "unique_id",
    "ID_REF": "id_ref",
    "VALUE": "value",
    "ABS_CALL": "abs_call",
    "DETECTION P-VALUE": "detection_p_value"
}


In [21]:
predicate_mapping = json.loads(predicate_mapping)

In [22]:
def generate_fol_predicates(row, predicate_mapping):
    """Convert a row of data into FOL predicates."""
    unique_id = row["Unique_ID"]  # Use Unique_ID as the main entity
    predicates = [f'GSM_Measurement({unique_id})']  # Main predicate

    # Convert each column into a predicate
    for col, predicate in predicate_mapping.items():
        value = row[col]
        formatted_value = value if pd.notna(value) else "null"

         # Exclude uniqueId(Unique_ID, Unique_ID)
        if predicate == "uniqueId" and formatted_value == unique_id:
            continue
        
        predicates.append(f'{predicate}({unique_id}, {formatted_value})')

    return ",\n    ".join(predicates) +","

In [23]:
# Generate predicates for each row
fol_predicates = [generate_fol_predicates(row, predicate_mapping) for _, row in sample_data.iterrows()]

# Print results
for pred in fol_predicates:
    print(pred)

GSM_Measurement(34838_GSM308246),
    unique_id(34838_GSM308246, 34838_GSM308246),
    id_ref(34838_GSM308246, 234976_x_at),
    value(34838_GSM308246, 1.21111),
    abs_call(34838_GSM308246, A),
    detection_p_value(34838_GSM308246, 0.80542),
GSM_Measurement(37395_GSM308246),
    unique_id(37395_GSM308246, 37395_GSM308246),
    id_ref(37395_GSM308246, 237533_at),
    value(37395_GSM308246, 1.36808),
    abs_call(37395_GSM308246, A),
    detection_p_value(37395_GSM308246, 0.888428),
GSM_Measurement(33285_GSM308246),
    unique_id(33285_GSM308246, 33285_GSM308246),
    id_ref(33285_GSM308246, 233421_s_at),
    value(33285_GSM308246, 1.01271),
    abs_call(33285_GSM308246, A),
    detection_p_value(33285_GSM308246, 0.753906),
GSM_Measurement(18008_GSM308246),
    unique_id(18008_GSM308246, 18008_GSM308246),
    id_ref(18008_GSM308246, 218582_at),
    value(18008_GSM308246, 85.563),
    abs_call(18008_GSM308246, P),
    detection_p_value(18008_GSM308246, 0.000244141),
GSM_Measurement(813

In [24]:
with open("./outputs/FOL_predicates.txt", "w") as file:
    file.write("\n".join(fol_predicates))

### Generating predicates that represent the relationship between GSE and GSM

In [25]:
gse.metadata

{'title': ['Hematopoietic Progenitor Cells of Different Donor Age'],
 'geo_accession': ['GSE12277'],
 'status': ['Public on Jun 12 2009'],
 'submission_date': ['Jul 29 2008'],
 'last_update_date': ['Mar 25 2019'],
 'pubmed_id': ['19513108'],
 'summary': ['In this series we have analyzed the effect of donor age on the gene expression profile of human hematopoietic stem and progenitor cells (HPC). Cells were taken from umbilical cord blood (CB) or from G-CSF mobilized blood of healthy donors for allogeneic blood stem cell transplantation.'],
 'overall_design': ['Hematopoietic progenitor cells (HPC) were isolated within the CD34+ cell fraction from fresh human cord blood (CB) or from G-CSF mobilized peripheral blood (PB) as described in detail before (Wagner et al., Blood, 2004, 104:675-684; Wagner et al., Stem cells, 2005, 23:1180-1191; Wagner et al., 2007, 10:2638-2657). Differential gene expression was subsequently compared in the CD34+ samples of different donor age.'],
 'type': ['Exp

In [26]:
def generate_fol_relationship(metadata):
    """
    Generates First-Order Logic (FOL) representation of the GSE-GSM relationship.

    Args:
        metadata (dict): Dictionary containing GSE metadata.

    Returns:
        str: FOL statements representing GSE-GSM relationships.
    """
    gse_id = metadata.get("geo_accession", ["Unknown_GSE"])[0]  # Get GSE ID
    gsm_ids = metadata.get("sample_id", [])  # Get GSM IDs list

    fol_statements = [f"has_sample({gse_id}, {gsm});" for gsm in gsm_ids]  # Create FOL statements

    return "\n".join(fol_statements)

In [27]:
gse_metadata= gse.metadata

In [28]:
fol_output = generate_fol_relationship(gse_metadata)

In [29]:
print("Example FOL Output:")
print(fol_output)

Example FOL Output:
has_sample(GSE12277, GSM308237);
has_sample(GSE12277, GSM308239);
has_sample(GSE12277, GSM308241);
has_sample(GSE12277, GSM308242);
has_sample(GSE12277, GSM308244);
has_sample(GSE12277, GSM308245);
has_sample(GSE12277, GSM308246);
has_sample(GSE12277, GSM308247);
has_sample(GSE12277, GSM308254);
has_sample(GSE12277, GSM308255);
has_sample(GSE12277, GSM308317);
has_sample(GSE12277, GSM308461);
has_sample(GSE12277, GSM308462);
has_sample(GSE12277, GSM308463);
has_sample(GSE12277, GSM308465);
has_sample(GSE12277, GSM308467);
has_sample(GSE12277, GSM308469);
has_sample(GSE12277, GSM308470);
has_sample(GSE12277, GSM308473);


### PLN formatted predicate generation

In [30]:
def declare_types(predicate_mapping):
    """Generate PLN type declarations based on predicate_mapping."""
    pln_types = [f"(: {pln_type} (-> Object Type))" for pln_type in predicate_mapping.values()]
    return "\n".join(pln_types)

In [ ]:
def declare_instances(df, predicate_mapping):
    """Generate PLN instance declarations with relationships to uniqueId."""
    pln_instances = []
    
    for _, row in df.iterrows():
        unique_id = row["Unique_ID"]  # Unique identifier for this row
        
        for col in df.columns:
            if col in predicate_mapping:  # Ensure only mapped columns are used
                mapped_col = predicate_mapping[col]
                instance = row[col]
                
                # Declare the instance type
                pln_instances.append(f"(: {instance} {mapped_col})")
        
        # pln_instances.append(f"\n") # Add newline for separation

        for col in df.columns:
            if col in predicate_mapping:  # Ensure only mapped columns are used
                mapped_col = predicate_mapping[col]
                instance = row[col]        
                
                if mapped_col == "uniqueId":
                    continue
                # Define relationship with uniqueId
                pln_instances.append(f"(: ({mapped_col} {unique_id} {instance}))")
        pln_instances.append(f"\n") # Add newline for separation
    
    return "\n".join(pln_instances)

In [32]:
type_declarations = declare_types(predicate_mapping)
print("Type Declarations:")
print(type_declarations)

Type Declarations:
(: unique_id (-> Object Type))
(: id_ref (-> Object Type))
(: value (-> Object Type))
(: abs_call (-> Object Type))
(: detection_p_value (-> Object Type))


In [33]:
PLN_predicates = declare_instances(sample_data, predicate_mapping)
print(PLN_predicates)

(: 34838_GSM308246 unique_id)
(: 234976_x_at id_ref)
(: 1.21111 value)
(: A abs_call)
(: 0.80542 detection_p_value)
(: (unique_id 34838_GSM308246 34838_GSM308246))
(: (id_ref 34838_GSM308246 234976_x_at))
(: (value 34838_GSM308246 1.21111))
(: (abs_call 34838_GSM308246 A))
(: (detection_p_value 34838_GSM308246 0.80542))


(: 37395_GSM308246 unique_id)
(: 237533_at id_ref)
(: 1.36808 value)
(: A abs_call)
(: 0.888428 detection_p_value)
(: (unique_id 37395_GSM308246 37395_GSM308246))
(: (id_ref 37395_GSM308246 237533_at))
(: (value 37395_GSM308246 1.36808))
(: (abs_call 37395_GSM308246 A))
(: (detection_p_value 37395_GSM308246 0.888428))


(: 33285_GSM308246 unique_id)
(: 233421_s_at id_ref)
(: 1.01271 value)
(: A abs_call)
(: 0.753906 detection_p_value)
(: (unique_id 33285_GSM308246 33285_GSM308246))
(: (id_ref 33285_GSM308246 233421_s_at))
(: (value 33285_GSM308246 1.01271))
(: (abs_call 33285_GSM308246 A))
(: (detection_p_value 33285_GSM308246 0.753906))


(: 18008_GSM308246 unique_id

In [34]:
with open("./outputs/PLN_predicates.txt", "w") as file:
    file.write(type_declarations + "\n\n" + PLN_predicates)

### Convert to metta atom

In [35]:
def declare_predicate_mapping(predicate_mapping):
    """Generate PLN row structure using predicate_mapping."""
    placeholders = " ".join(f"${value}" for key, value in predicate_mapping.items() if key != "Unique_ID")  # Exclude uniqueId
    mappings = " ".join(f"({value} ${value})" for key, value in predicate_mapping.items() if key != "Unique_ID")  # Exclude uniqueId

    return f"""(= (row $uniqueId {placeholders})
    (add-atom &self ($uniqueId ({mappings})))
)\n"""


In [36]:
def declare_instances(df, predicate_mapping):
    """Generate PLN instance declarations for each row."""
    pln_instances = []
    
    for _, row in df.iterrows():
        values = " ".join(str(row[col]) for col in predicate_mapping if col in df.columns)  # Extracting values
        pln_instances.append(f"!(row {values})")
    
    return "\n".join(pln_instances)

In [37]:
row_definition= declare_predicate_mapping(predicate_mapping)
print(row_definition)

(= (row $uniqueId $id_ref $value $abs_call $detection_p_value)
    (add-atom &self ($uniqueId ((id_ref $id_ref) (value $value) (abs_call $abs_call) (detection_p_value $detection_p_value))))
)



In [38]:
instances= declare_instances(sample_data, predicate_mapping)
print(instances)

!(row 34838_GSM308246 234976_x_at 1.21111 A 0.80542)
!(row 37395_GSM308246 237533_at 1.36808 A 0.888428)
!(row 33285_GSM308246 233421_s_at 1.01271 A 0.753906)
!(row 18008_GSM308246 218582_at 85.563 P 0.000244141)
!(row 8131_GSM308246 208572_at 3.35546 A 0.432373)
!(row 36478_GSM308246 236616_at 16.7406 A 0.19458)
!(row 25769_GSM308246 225899_x_at 181.771 P 0.000732422)
!(row 27703_GSM308246 227836_at 83.1577 P 0.000244141)
!(row 9698_GSM308246 210153_s_at 73.9066 P 0.000244141)
!(row 43671_GSM308246 243809_at 0.524351 A 0.72583)
!(row 44257_GSM308246 244395_at 12.599 P 0.0107422)
!(row 47017_GSM308246 1555284_at 11.9666 P 0.0185547)
!(row 54118_GSM308246 1569676_at 4.07512 A 0.366211)
!(row 8188_GSM308246 208630_at 396.286 P 0.00195313)
!(row 25503_GSM308246 225633_at 188.004 P 0.000244141)
!(row 49195_GSM308246 1558747_at 15.9805 P 0.00805664)
!(row 43531_GSM308246 243669_s_at 1.6813 A 0.969727)
!(row 40083_GSM308246 240221_at 30.4075 P 0.0012207)
!(row 30931_GSM308246 231064_s_at 1.7

In [39]:
with open("./outputs/Metta.txt", "w") as file:
    file.write(row_definition + "\n\n" + instances)